In [2]:
%%bash
export host=database-1.cluster-czfb3bjo566o.ap-northeast-1.neptune.amazonaws.com
export port=8182

In [92]:
import neptune_ml_utils as neptune_ml
neptune_ml.check_ml_enabled()

This Neptune cluster is configured to use Neptune ML


In [93]:
neptune_ml.load_configuration()

('database-1.cluster-czfb3bjo566o.ap-northeast-1.neptune.amazonaws.com',
 8182,
 False)

In [94]:
s3_bucket_uri="s3://ai-datalab-sunbiao/nepdata/"
# remove trailing slashes
s3_bucket_uri = s3_bucket_uri[:-1] if s3_bucket_uri.endswith('/') else s3_bucket_uri

setup_node_classification=True
setup_node_regression=False
setup_link_prediction=False
setup_edge_classification=False
setup_edge_regression=False

endpoints=neptune_ml.setup_pretrained_endpoints(s3_bucket_uri, setup_node_classification, setup_node_regression, setup_link_prediction, setup_edge_classification, setup_edge_regression)

node_classification_endpoint=endpoints['node_classification_endpoint_name']['EndpointName']

In [91]:
%%gremlin

g.V().where(__.hasNot('address_type')).group().by(both().count()).by(count())

In [26]:
%%gremlin 
g.V().and(__.hasNot('address_type'),__.both().count().is(893)).valueMap(true).limit(10)

In [9]:
my_node_labels_b = '{"vertex":"name"}'
my_edge_labels_b = '{"payto":"tx_amt"}'

In [130]:
%%gremlin -p v,outv -d $my_node_labels_b 
g.V('TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX')
 .repeat(both())
 .until(outE().count().is(0))
 .path().by(T.id).limit(1000)

In [31]:
%%gremlin -p v,oute,inv  -de $my_edge_labels_b
g.V('TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX').outE().inV().path().by(valueMap(true)).limit(1000)

In [1]:
%%gremlin
g.V('TQBPvuuUTXpqM5avu6Kt1crnT4DwC18Uax').properties("address_type").value()

In [25]:
%%gremlin
g.V().has('address_type').groupCount().by('address_type').unfold().order().by(keys)

In [208]:
%%gremlin
g.V().groupCount().by(label).unfold().order().by(keys)

In [209]:
export_params={ 
"command": "export-pg", 
"params": { "endpoint": neptune_ml.get_host(),
            "profile": "neptune_ml",
            "useIamAuth": neptune_ml.get_iam(),
            "cloneCluster": False
            }, 
"outputS3Path": f'{s3_bucket_uri}/neptune-export',
"additionalParams": {
        "neptune_ml": {
          "version": "v2.0",
          "targets": [
            {
              "node": "cryp_add",
              "property": "address_type",
              "type": "classification"
            }
          ],
         "features": [
         {
                "node": "Address",
                "property": "name",
                "type": "text_word2vec"
            }
         ]
        }
      },
"jobSize": "medium"}

这里把neptune里的vertex数据删除掉，用来作为训练数据，导出后再恢复

In [98]:
%%gremlin
g.V().hasLabel('vertex').drop()

In [61]:
%%gremlin
g.V().groupCount().by(label).unfold().order().by(keys)

In [103]:
%%gremlin
g.V('TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX').valueMap(true)

In [210]:
%%neptune_ml export start --export-url evdwysbwg3.execute-api.ap-northeast-1.amazonaws.com/v1 --export-iam --wait --store-to export_results
${export_params}

Output()

In [211]:
# The training_job_name can be set to a unique value below, otherwise one will be auto generated
training_job_name=neptune_ml.get_training_job_name('node-classification')
print(training_job_name)
processing_params = f"""
--config-file-name training-data-configuration.json
--job-id {training_job_name} 
--s3-input-uri {export_results['outputS3Uri']} 
--s3-processed-uri {str(s3_bucket_uri)}/preloading """

node-classification-1636723249


In [212]:
%neptune_ml dataprocessing start --wait --store-to processing_results {processing_params}

Output()

In [213]:
training_params=f"""
--job-id {training_job_name}
--data-processing-id {training_job_name} 
--instance-type ml.g4dn.2xlarge
--s3-output-uri {str(s3_bucket_uri)}/training
--max-hpo-number 2
--max-hpo-parallel 2 """

In [214]:
%neptune_ml training start --wait --store-to training_results {training_params}

Output()

In [216]:
endpoint_params=f"""
--job-id {training_job_name} 
--model-training-job-id {training_job_name} """

In [217]:
#
%neptune_ml endpoint create --wait --store-to endpoint_results {endpoint_params}

Output()

In [110]:
#这个和上面的create方法都可以
%%bash
curl \
  -X POST https://database-1.cluster-czfb3bjo566o.ap-northeast-1.neptune.amazonaws.com:8182/ml/endpoints \
  -H 'Content-Type: application/json' \
  -d '{
        "id" : "sunbiao-endpoint-202111121413",
        "mlModelTrainingJobId": "node-classification-1636695309"
      }'

{"id":"sunbiao-endpoint-202111121313"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   160  100    38  100   122     20     64  0:00:01  0:00:01 --:--:--    84


In [218]:
endpoint = endpoint_results['endpoint']['name']
print(endpoint)

node-cla-2021-11-12-13-41-2310000-endpoint


In [2]:
endpoint = "node-cla-2021-11-12-13-41-2310000-endpoint"

ok，首先drop一个exchange 的属性查看是否能够得出正确结果

In [ ]:
%%gremlin
g.V().where(__.outE().count().is(gt(200))).limit(10).valueMap()

In [225]:
%%gremlin 
g.V().and(__.hasNot('address_type'),__.inE().count().is(gt(2000))).valueMap(true).limit(10)

In [81]:
%%gremlin
g.V().where(__.label().count().is(1)).limit(10).valueMap(true)

In [155]:
%%gremlin
g.V('TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX').properties("address_type").drop()
g.V('TAUN6FwrnwwmaEqYcckffC7wYmbaS6cBiX').valueMap()

In [3]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
  V('TYbT3dBgQMfKyb4BpDAdHqtF1zteayQVeF').properties("address_type").with("Neptune#ml.classification").value()

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
    with("Neptune#ml.limit", 2).
    V().hasNot('address_type').
    project('name', 'predicted_type').
        by('name').
        by(properties("address_type").with("Neptune#ml.classification").value().fold()).
    order(local).by(keys, desc)

In [4]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
  with("Neptune#ml.threshold", 0.7D).
    V().and(__.hasNot('address_type'),__.properties("address_type").with("Neptune#ml.classification").value().is('Exchange')).valueMap(true).limit(20)

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
  V('TFp13UQaBqSBEGEUmD7LmFBHc3AuGPgtRH').properties("address_type").with("Neptune#ml.classification").value()

In [245]:
%%gremlin -p v,ine,outv  -de $my_edge_labels_b
g.V('TFp13UQaBqSBEGEUmD7LmFBHc3AuGPgtRH').inE().outV().path().by(valueMap(true)).limit(1000)

In [52]:
%%bash
cat ~/.bashrc

# .bashrc

# Source global definitions
if [ -f /etc/bashrc ]; then
	. /etc/bashrc
fi

# User specific aliases and functions

# >>> conda initialize >>>
# !! Contents within this block are managed by 'conda init' !!
__conda_setup="$('/home/ec2-user/anaconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/home/ec2-user/anaconda3/etc/profile.d/conda.sh" ]; then
        . "/home/ec2-user/anaconda3/etc/profile.d/conda.sh"
    else
        export PATH="/home/ec2-user/anaconda3/bin:$PATH"
    fi
fi
unset __conda_setup
# <<< conda initialize <<<

export GRAPH_NOTEBOOK_AUTH_MODE=DEFAULT
export GRAPH_NOTEBOOK_HOST=database-1.cluster-czfb3bjo566o.ap-northeast-1.neptune.amazonaws.com
export GRAPH_NOTEBOOK_PORT=8182
export NEPTUNE_LOAD_FROM_S3_ROLE_ARN=
export AWS_REGION=ap-northeast-1


In [117]:
%%gremlin
g.V().where(__.hasNot('address_type')).limit(100).valueMap(true)

In [207]:
%%bash

curl -X POST \
    -H 'Content-Type: application/json' \
    https://database-1.cluster-czfb3bjo566o.ap-northeast-1.neptune.amazonaws.com:8182/loader -d '
    {
      "source" : "s3://ai-datalab-sunbiao/nepdata/neponelabel",
      "format" : "csv",
      "iamRoleArn" : "arn:aws:iam::240778039237:role/neptunes3",
      "region" : "ap-northeast-1",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "FALSE",
      "queueRequest" : "TRUE",
      "dependencies" : []
    }'

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "6824910f-3ccf-46c2-9a0a-3779a9b35a32"
    }
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   483  100   110  100   373    447   1516 --:--:-- --:--:-- --:--:--  1963


In [124]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

In [206]:
%%gremlin
g.V().drop()